In [1]:
import numpy as np
from numpy import linalg
import cvxopt
import cvxopt.solvers
import pandas as pd
from sklearn import cross_validation
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
from sklearn import svm

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv("pima-indians-diabetes.csv")
train.head()

,1,2,3,4,5,6,7,8,9
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
train['9'] = train['9'].map({1: 1, 0: -1})
print(train['9'])

0      1
1     -1
2      1
3     -1
4      1
5     -1
6      1
7     -1
8      1
9      1
10    -1
11     1
12    -1
13     1
14     1
15     1
16     1
17     1
18    -1
19     1
20    -1
21    -1
22     1
23     1
24     1
25     1
26     1
27    -1
28    -1
29    -1
      ..
738   -1
739    1
740    1
741   -1
742   -1
743    1
744   -1
745   -1
746    1
747   -1
748    1
749    1
750    1
751   -1
752   -1
753    1
754    1
755    1
756   -1
757    1
758   -1
759    1
760   -1
761    1
762   -1
763   -1
764   -1
765   -1
766    1
767   -1
Name: 9, Length: 768, dtype: int64


In [4]:

train=np.asarray(train)

In [5]:
min_train=np.zeros((268,9))
max_train=np.zeros((500,9))
min_train=np.asarray(min_train)
max_train=np.asarray(max_train)

In [6]:

k=0
l=0
for i in range(0,768):
    if(train[i][8]==1):
        for j in range(0,9):
            min_train[k][j]=train[i][j]   
        k=k+1
    else :
        for j in range(0,9):
            max_train[l][j]=train[i][j]
        l=l+1

In [7]:
print(min_train)

[[  6.    148.     72.    ...   0.627  50.      1.   ]
 [  8.    183.     64.    ...   0.672  32.      1.   ]
 [  0.    137.     40.    ...   2.288  33.      1.   ]
 ...
 [  6.    190.     92.    ...   0.278  66.      1.   ]
 [  9.    170.     74.    ...   0.403  43.      1.   ]
 [  1.    126.     60.    ...   0.349  47.      1.   ]]


#768/5 =153.6
#So we have to make no. of instances in data1,2 as 153 and data3,4,5 as 154 
#Also min class value will be 35% of 153=53 and max class=100

In [8]:
data1=np.zeros((153,9))
data2=np.zeros((153,9))
data3=np.zeros((154,9))
data4=np.zeros((154,9))
data5=np.zeros((154,9))

In [9]:
for i in range(0,268):
    for j in range(0,9):
        if(i<53):
            data1[i][j]=min_train[i][j]
        elif(52<i and i<106):
            data2[i-53][j]=min_train[i][j]
        elif(105<i and i<160):
            data3[i-106][j]=min_train[i][j]
        elif(159<i and i<214):
            data4[i-160][j]=min_train[i][j]
        elif(213<i and i<268):
            data5[i-213][j]=min_train[i][j]
    
        
print(data5[53])

[  9.    170.     74.     31.      0.     44.      0.403  43.      1.   ]


In [10]:
for i in range(0,500):
    for j in range(0,9):
        if(i<100):
            data1[i+53][j]=max_train[i][j]
        elif(99<i and i<200):
            data2[i-47][j]=max_train[i][j]
        elif(199<i and i<300):
            data3[i-146][j]=max_train[i][j]
        elif(299<i and i<400):
            data4[i-246][j]=max_train[i][j]
        elif(399<i and i<500):
            data5[i-346][j]=max_train[i][j]   

In [12]:
count_test=0
for i in range(0,154):
    if(y_train[i]==1):
        count_test=count_test+1
print("Min",count_test)
print("Max",154-count_test)
print("Ratio",count_test/(154-count_test))

NameError: name 'y_train' is not defined

In [4]:
features = train.columns[0:8]
X = train[features]
y = train['1']
X.head()

,6,148,72,35,0,33.6,0.627,50
0,1,85,66,29,0,26.6,0.351,31
1,8,183,64,0,0,23.3,0.672,32
2,1,89,66,23,94,28.1,0.167,21
3,0,137,40,35,168,43.1,2.288,33
4,5,116,74,0,0,25.6,0.201,30


In [5]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,y,test_size=0.2,random_state=50)


In [6]:
X_train=np.asarray(X_train)
y_train=np.asarray(y_train)

In [11]:
C=0.1
clf_svm = svm.SVC(kernel='rbf',gamma =0.0001, C=C)
clf_svm.fit(X_train, y_train)
y_pred_svm = clf_svm.predict(X_test) 
acc_svm = accuracy_score(y_test, y_pred_svm)
print ("Overall Linear SVM accuracy: ",acc_svm)

Overall Linear SVM accuracy:  0.7142857142857143


In [19]:
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma=0.001):
    return np.exp(-linalg.norm(x-y)**2 / (2 * (sigma ** 2)))

In [24]:
from cvxopt import matrix
class SVM(object):

    def __init__(self, kernel=gaussian_kernel, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)
    def fit(self, X, y):
        n_samples, n_features = X.shape
        # Gram matrix
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = self.kernel(X[i], X[j])
               # print(K[i,j])
        print(K.shape)

        P = cvxopt.matrix(np.outer(y,y) * K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
       # print(a)
        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        #print(sv)
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X[sv]
        self.sv_y = y[sv]
        print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * K[ind[n],sv])
        self.b /= len(self.a)

        # Weight vector
        if self.kernel == gaussian_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
                #print(self.w)
        else:
            self.w = None

    def project(self, X):
        if self.w is not None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * self.kernel(X[i], sv)
                y_predict[i] = s
              #  print(y_predict[i])
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))

In [25]:

if __name__ == "__main__":
    import pylab as pl   
    def test_non_linear():
        
        clf = SVM(C=100.0)
        clf.fit(X_train, y_train)

        y_predict = clf.predict(X_test)
        correct = np.sum(y_predict == y_test)
        print("%d out of %d predictions correct" % (correct, len(y_predict)))
        print(current/len(y_predict))
        
    def test_soft():

        clf = SVM(C=100.0)
        clf.fit(X_train, y_train)

        y_predict = clf.predict(X_test)
        correct = np.sum(y_predict == y_test)
        print("%d out of %d predictions correct" % (correct, len(y_predict)))
        print(correct/len(y_predict))

    test_soft()
    #test_non_linear()
    

(613, 613)
     pcost       dcost       gap    pres   dres
 0:  2.9827e+05 -8.0785e+05  1e+06  1e-11  1e-14
 1:  6.1640e+04 -8.4657e+04  1e+05  1e-11  4e-15
 2:  7.8898e+03 -1.3304e+04  2e+04  2e-12  9e-16
 3:  6.0083e+02 -2.3130e+03  3e+03  2e-14  1e-15
 4: -2.2511e+02 -6.0023e+02  4e+02  4e-13  2e-16
 5: -2.7924e+02 -3.0587e+02  3e+01  7e-14  1e-16
 6: -2.7978e+02 -2.8009e+02  3e-01  2e-13  2e-17
 7: -2.7978e+02 -2.7978e+02  3e-03  1e-13  2e-17
 8: -2.7978e+02 -2.7978e+02  3e-05  8e-14  3e-17
Optimal solution found.
optimal
613 support vectors out of 613 points
51 out of 154 predictions correct
0.33116883116883117
